In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
import pypfopt
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import risk_models
import streamlit as st

In [16]:
tickers = ['TSM','AAPL','AMD','BABA', 'SPY', 'V','QQQ']

In [17]:
data =yf.download(tickers, start='2020-01-01', end='2021-12-31')

[*********************100%***********************]  7 of 7 completed


In [18]:
data.head()

Adj Close                                                 \
                 AAPL        AMD        BABA         QQQ         SPY   
Date                                                                   
2020-01-02  73.785919  49.099998  219.770004  213.641678  312.395050   
2020-01-03  73.068565  48.599998  217.000000  211.684708  310.029480   
2020-01-06  73.650795  48.389999  216.639999  213.048630  311.212311   
2020-01-07  73.304413  48.250000  217.630005  213.018982  310.337280   
2020-01-08  74.483597  47.830002  218.000000  214.620132  311.991180   

                                       Close                         ...  \
                  TSM           V       AAPL        AMD        BABA  ...   
Date                                                                 ...   
2020-01-02  56.990185  188.125137  75.087502  49.099998  219.770004  ...   
2020-01-03  55.110767  186.628952  74.357498  48.599998  217.000000  ...   
2020-01-06  54.474796  186.225403  74.949997  48.389999  216.639999  ...   
2020-01-07  55.357555  185.733231  74.597504  48.250000  217.630005  ...   
2020-01-08  55.765709  188.912613  75.797501  47.830002  218.000000  ...   

                  Open                            Volume                      \
                   SPY        TSM           V       AAPL       AMD      BABA   
Date                                                                           
2020-01-02  323.540009  59.599998  189.000000  135480400  80331100  15873500   
2020-01-03  321.160004  58.970001  188.410004  146322800  73127400   8604500   
2020-01-06  320.489990  57.599998  188.000000  118387200  47934900  11885500   
2020-01-07  323.019989  57.450001  189.589996  108872000  58061400   9388000   
2020-01-08  322.940002  58.189999  189.490005  132079200  53767000  11959100   

                                                    
                 QQQ       SPY       TSM         V  
Date                                                
2020-01-02  30969400  59151200   8432600   8733000  
2020-01-03  27518900  77709700  10546400   4899700  
2020-01-06  21655300  55653900   8897200  10109500  
2020-01-07  22139300  40496400   7444300   4392300  
2020-01-08  26397300  68296000   5381500   5712000  

[5 rows x 42 columns]

In [19]:
data2 = data['Adj Close']

In [20]:
mu =pypfopt.expected_returns.mean_historical_return(data2)

In [64]:
data2.index

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2021-12-16', '2021-12-17', '2021-12-20', '2021-12-21',
               '2021-12-22', '2021-12-23', '2021-12-27', '2021-12-28',
               '2021-12-29', '2021-12-30'],
              dtype='datetime64[ns]', name='Date', length=504, freq=None)

In [44]:
cov =pypfopt.risk_models.sample_cov(data2)
w = np.array([0,.9,0,0,0,.05,.08])

In [45]:
cov

,AAPL,AMD,BABA,QQQ,SPY,TSM,V
AAPL,0.140659,0.118143,0.056898,0.093483,0.075901,0.083030,0.079917
AMD,0.118143,0.275616,0.068735,0.105688,0.080416,0.127110,0.081295
BABA,0.056898,0.068735,0.182677,0.053732,0.042252,0.055270,0.047043
QQQ,0.093483,0.105688,0.053732,0.080224,0.066057,0.076522,0.071551
SPY,0.075901,0.080416,0.042252,0.066057,0.064389,0.063877,0.073893
TSM,0.083030,0.127110,0.055270,0.076522,0.063877,0.158501,0.066570
V,0.079917,0.081295,0.047043,0.071551,0.073893,0.066570,0.120471


In [46]:
w


array([0.  , 0.9 , 0.  , 0.  , 0.  , 0.05, 0.08])

In [47]:
w_min =np.array([-.1,-.2,-.2,-.10,.2,-.05,-.05])

In [48]:
rf =.03

In [59]:
ef = EfficientFrontier(mu, cov,weight_bounds=(0,1))
ef.add_constraint(lambda x: x <= w)
ef.add_constraint(lambda y: y >= w_min)
ef.add_constraint(lambda y: y >= w_min)
ef.add_constraint(lambda y: y >= w_min)

In [50]:
sharpe =ef.max_sharpe(risk_free_rate=rf)
clean_weights  = ef.clean_weights()

In [51]:
print(clean_weights)

OrderedDict([('AAPL', 0.0), ('AMD', 0.9), ('BABA', 0.0), ('QQQ', 0.0), ('SPY', 0.0), ('TSM', 0.05), ('V', 0.05001)])


In [28]:
ef.portfolio_performance(risk_free_rate= rf, verbose=True)

Expected annual return: 49.6%
Annual volatility: 32.7%
Sharpe Ratio: 1.43


(0.49625477892750863, 0.32698184865018204, 1.4259347448559025)

In [29]:
ef2 = EfficientFrontier(mu, cov,weight_bounds=(0,1))
ef2.add_constraint(lambda x: x <= w)
# ef2.add_constraint(lambda x: x >= w_min)


In [58]:
w

array([0.  , 0.9 , 0.  , 0.  , 0.  , 0.05, 0.08])

In [62]:
for t in w:
        if t == 0:
                ef.add_constraint(lambda x: x >= t)
        else:
                ef.add_constraint(lambda y: y <= t)


In [30]:
maxr = ef2.efficient_risk(.3,market_neutral=True)

In [31]:
maxr

OrderedDict([('AAPL', -0.0500000000391205),
             ('AMD', 0.2499999999861249),
             ('BABA', -0.2000000000009474),
             ('QQQ', -0.0999999999865808),
             ('SPY', 0.1999999999987815),
             ('TSM', -0.0499999999586631),
             ('V', -0.0499999999995998)])

In [32]:
ef2.portfolio_performance(risk_free_rate=rf, verbose=True)

Expected annual return: 18.6%
Annual volatility: 12.4%
Sharpe Ratio: 1.26


(0.18623442311723853, 0.12361507874315003, 1.2638783610037223)

In [63]:
from cvxpy import SCS